In [ ]:
from scripts.data_prep import process_event_batches
import os, re
import torch
import numpy as np
import copy as copy
import time as time
from pathlib import Path

In [ ]:
ruta = 'data/events' #ruta a la carpeta que contiene los eventos
file_names = [f for f in os.listdir(ruta) if os.path.isfile(os.path.join(ruta, f))]
nums = list(dict.fromkeys(m.group(1) for f in file_names if (m := re.match(r"^event(\d+)-.*\.csv$", f))))


# Generador del dataset con filtro ideal (pt_min = 0.5)

In [ ]:
import os
from tqdm import tqdm



def create_dataset(event_names, output_dir, pt_min, pt_max, dtheta, dphi):
    os.makedirs(output_dir, exist_ok=True)

    for ev_id in tqdm(event_names, desc="Creando dataset"):
        graphs, _ = process_event_batches(ev_id, drop_y=False, get_true_tracks=False,
                                          dtheta=dtheta, dphi=dphi,
                                          pt_min=pt_min, pt_max=pt_max)

        for i, g in enumerate(graphs):
            filename = f"{ev_id}_graph_{i}.pt"
            torch.save(g, os.path.join(output_dir, filename))
            

events = nums[0:10]


ruta_save = 'data/dataset_graphs_ideal'
create_dataset(events, ruta_save, 0.5, 1000, dtheta=np.pi*2, dphi=np.pi)

# Generador con filtro real (pt_min = 0, el filtro se encarga de distinguir)

In [ ]:
events = nums[0:10]
create_dataset(events, 'dataset_graphs_real', 0, 1000, dtheta=np.pi*2, dphi=np.pi)


In [ ]:
#Para conocer la distribución entre las dos clases de hits de un grafo generado y guardado en disco en la ruta train_dir
train_dir = 'data/dataset_graphs_real/test'

pt_files = list(Path(train_dir).glob("*.pt"))
if not pt_files:
    raise RuntimeError(f"No hay archivos .pt en {train_dir}")
file_path = pt_files[3]  
g = torch.load(str(file_path), weights_only=False)

y = g.y 
num_pos = int((y == 1).sum().item())
num_neg = int((y == 0).sum().item())

ratio = num_neg / num_pos if num_pos > 0 else float('inf')
print(f"Archivo: {file_path.name}")
print(f"Positivos (True): {num_pos}")
print(f"Negativos (False): {num_neg}")
print(f"Proporción False:True = {num_neg}:{num_pos} ≃ {ratio:.3f}:1")
